# Installing Enzyme

Tutorial tested with Julia 1.7-beta3

Using the Julia package manger:
```julia
import Pkg
Pkg.add("Enzyme")
```

In [8]:
import Pkg
Pkg.activate(; temp=true)
Pkg.add(Pkg.PackageSpec(name="Enzyme", rev="065224c0efa2d00d25605ec2aa8cb869440ee7ae"))
Pkg.add("BenchmarkTools")
Pkg.add("ForwardDiff")

  Activating new project at `/tmp/jl_BIgUFY`
  Activating new project at `/tmp/jl_TL5neI`
  Activating new project at `/tmp/jl_OZs6yZ`
  Activating new project at `/tmp/jl_06qQBI`
   Resolving package versions...
    Updating `/tmp/jl_06qQBI/Project.toml`
  [7da242da] + Enzyme v0.7.0 `https://github.com/wsmoses/Enzyme.jl.git#065224c`
    Updating `/tmp/jl_06qQBI/Manifest.toml`
  [79e6a3ab] + Adapt v3.3.1
  [fa961155] + CEnum v0.4.1
  [7da242da] + Enzyme v0.7.0 `https://github.com/wsmoses/Enzyme.jl.git#065224c`
  [e2ba6199] + ExprTools v0.1.6
  [61eb1bfa] + GPUCompiler v0.13.7
  [692b3bcd] + JLLWrappers v1.3.0
  [929cbde3] + LLVM v4.6.0
  [d8793406] + ObjectFile v0.3.7
  [21216c6a] + Preferences v1.2.2
  [189a3867] + Reexport v1.2.2
  [53d494c1] + StructIO v0.3.0
  [a759f4b9] + TimerOutputs v0.5.13
  [7cc45869] + Enzyme_jll v0.0.21+0
  [dad2f222] + LLVMExtra_jll v0.0.11+0
  [0dad84c5] + ArgTools
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + Dates
  [f43a241f] + Downloads

In [9]:
using Enzyme

# Activity annotations
- `Const`
- `Active`
- `Duplicated`
- `DuplicatedNoNeed`

In [ ]:
square(x) = x^2

In [ ]:
autodiff(square, 1.0)

Default activity for values is `Const`

In [ ]:
autodiff(square, Const(1.0))

In [ ]:
autodiff(square, Active(1.0))

## Supporting mutating functions

Enzyme can differentiate through mutating functions. This requires that the users passes in the shadow variables with the `Duplicated` or `DuplicatedNoNeed` activity annotation.

In [ ]:
function cube(y, x)
	y[] = x[]^3
	return nothing
end

In [ ]:
let
	x = Ref(4.0)
	y = Ref(0.0)
	cube(y, x)
	y[]
end


In order to calculate the gradient of `x`, we have to propagate `1.0` into the
shadow `dy`.


In [ ]:

x = Ref(4.0)
dx = Ref(0.0)

y = Ref(0.0)
dy = Ref(1.0)

autodiff(cube, Duplicated(y, dy), Duplicated(x, dx))
y[], dy[], x[], dx[]

# Reflection

In [ ]:

Enzyme.Compiler.enzyme_code_llvm(cube,
	Tuple{Enzyme.Duplicated{Base.RefValue{Float64}}, 
	Duplicated{Base.RefValue{Float64}}}, debuginfo=:none)



# Differentiating through control-flow
Let's differentiate through some control flow. This kind of scalar code is where normally one would use `ForwardDiff.jl` since the machine learning optimized toolkits like Zygote have unacceptable overheads.

In [ ]:
# Taylor series for `-log(1-x)`
# eval at -log(1-1/2) = -log(1/2)
function taylor(f::T, N=10^7) where T
    g = zero(T)
    for i in 1:N
        g += f^i / i
    end
    return g
end

autodiff(taylor, Active(0.5), Const(10^8))


In [ ]:
fwd_taylor(x) = ForwardDiff.derivative(taylor, 0.5)

enz_taylor(x) = autodiff(taylor, Active(x))


In [ ]:

@benchmark fwd_taylor($(Ref(0.5))[])

In [ ]:

@benchmark enz_taylor($(Ref(0.5))[])


# Differentiating through more complicated codes

## A custom matrix multiply

In [ ]:

function mymul!(R, A, B)
    @assert axes(A,2) == axes(B,1)
    @inbounds @simd for i in eachindex(R)
        R[i] = 0
    end
    @inbounds for j in axes(B, 2), i in axes(A, 1)
        @inbounds @simd for k in axes(A,2)
            R[i,j] += A[i,k] * B[k,j]
        end
    end
    nothing
end

In [ ]:
A = rand(1024, 64)
B = rand(64, 512)

R = zeros(size(A,1), size(B,2))
∂z_∂R = rand(size(R)...)  # Some gradient/tangent passed to us

∂z_∂A = zero(A)
∂z_∂B = zero(B)

Enzyme.autodiff(mymul!, 
	Duplicated(R, ∂z_∂R),
	Duplicated(A, ∂z_∂A),
	Duplicated(B, ∂z_∂B))


Let's confirm correctness of result

In [ ]:
R ≈ A * B

and correctness of the gradients

In [ ]:
∂z_∂A ≈ ∂z_∂R * B'

In [ ]:
# Some more fun

In [ ]:

struct LList
    next::Union{LList,Nothing}
	val::Float64
end 

function sumlist(n::LList)
    sum = 0.0
    while n !== nothing
        sum += n.val
        n = n.next
    end
    sum
end

In [ ]:

regular = LList(LList(nothing, 1.0), 2.0)
shadow  = LList(LList(nothing, 0.0), 0.0)
autodiff(sumlist, Duplicated(regular, shadow))

In [ ]:
shadow.val ≈ 1.0

In [ ]:
shadow.next.val ≈ 1.0